In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
import numpy as np
import time

# Iniciar random
np.random.seed(6)

# Variables globales con información para acelerar procesamiento
g_filas = g_cols = 0

ITER = 40
APRN = 0.5
LMBD = 0.000002

# Funciones

In [2]:
# linea es un string de la forma:
# "n0, n1, n2, ..., n10, r" donde nX es una propiedad del tráfico y r es
# su clasificación (1 si es botnet, 0 si no lo es)
def formato_inicial(linea):
    aux   = [float(i) for i in linea.split(',')]
    datos = np.array(aux[:-1])
    resul = aux[-1]
    return (datos, resul)

# Devuelve un RDD de los datos del fichero fn. La lectura con Spark
# ya es mucho más rápida que leyendo con numpy.loadtxt()
def readFile(fn):
    global g_filas, g_cols
    sc      = SparkContext("local[*]", "BotnetParalelaPruebas")
    todo    = sc.textFile(fn)
    ret     = todo.map(formato_inicial)
    g_filas = todo.count()
    g_cols  = len(ret.take(1)[0][0])
    return ret

In [3]:
def norm_tupla(tuplaXy):
    media = np.mean(tuplaXy[0])
    stdev = np.std(tuplaXy[0])
    tnorm = (tuplaXy[0] - media)/stdev
    return (tnorm, tuplaXy[1])

# Toma por parámetro un RDD de 1.000.000 de registros.
# Cada registro tiene la forma (X, y)
# X es un array con 11 flotantes
# y es 1 ó 0
# Devuelve un RDD equivalente donde la X de cada tupla está
# reescalada a N(0, 1) (media = 0, desv. est. = 1)
def normalizar(rdd):
    return rdd.map(norm_tupla)

In [4]:
# pesos: Array de (1Mx1)
# sesgo: flotante
# datos: RDD de 1M de regs. Cada reg: tupla (X, y) donde X es un array de (11x1)
# datos x sesgo: Array de (1Mx1)
# La multiplicación se puede hacer por filas y no se altera el resultado
def sigmoide(pesos, sesgo, datos):
    entrada = datos.map(lambda fila: np.matmul(fila[0], pesos) + sesgo)
    salida  = entrada.map(lambda x: 1/(1 + np.exp(-x)))
    return salida

def entrenar(datos, iteraciones, aprendizaje, tasareg):
    f = g_filas # 1.000.000
    c = g_cols  # 11
    pesos   = np.random.random([c, ])
    d_pesos = np.zeros([c, ])
    sesgo   = 0
    d_sesgo = 0
    for it in range(iteraciones):
        pred = sigmoide(pesos, sesgo, datos)
        for j in range(c):
            pass

# Pruebas

In [5]:
t1  = time.time()
rdd = readFile("../../../datos/botnet_tot_syn_l.csv")
t2  = time.time()
print("Tiempo transcurrido: {} segundos.".format(t2 - t1))

Tiempo transcurrido: 3.833125352859497 segundos.


In [6]:
t1 = time.time()
datos = normalizar(rdd)
t2 = time.time()
print("Tiempo transcurrido: {} segundos.".format(t2 - t1))
# Haz la media de uno de los X. Tiene que ser ~0
# print(np.mean(datos.take(1)[0][0])) # 0.000000000000000010092936587501423
# La desviación estándar tiene que ser ~1
# print(np.std(datos.take(1)[0][0])) # 1.0

Tiempo transcurrido: 5.316734313964844e-05 segundos.
1.0092936587501423e-17
1.0


In [7]:
t1 = time.time()
entrenar(datos, ITER, APRN, LMBD)
t2 = time.time()
print("Tiempo transcurrido: {} segundos.".format(t2 - t1))

Tiempo transcurrido: 0.00030994415283203125 segundos.
